code from [here](https://blog.roboflow.com/how-to-use-segment-anything-model-sam/)

In [4]:
#Load the model
import torch
from segment_anything import sam_model_registry
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q roboflow supervision 


DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
CHECKPOINT_PATH = "sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2445M  100 2445M    0     0  10.7M      0  0:03:47  0:03:47 --:--:-- 11.2M9M      0  0:03:44  0:00:29  0:03:15 11.2M     0  0:03:44  0:00:43  0:03:01 10.7M   0  10.9M      0  0:03:43  0:01:01  0:02:42 11.2M  703M    0     0  10.8M      0  0:03:44  0:01:04  0:02:40 10.6M 0  10.9M      0  0:03:44  0:01:07  0:02:37 11.2M0.9M      0  0:03:44  0:01:22  0:02:22 10.7M  10.9M      0  0:03:44  0:01:25  0:02:19 10.7M 1147M    0     0  10.7M      0  0:03:46  0:01:46  0:02:00 10.9M     0  10.8M      0  0:03:46  0:01:59  0:01:47 11.2M    0  0:03:46  0:02:09  0:01:37 11.2M     0  0:03:46  0:02:17  0:01:29 11.2M  0:03:45  0:02:27  0:01:18 11.2M 1706M    0     0  10.8M      0  0:03:45  0:02:37  0:01:08 11.2M     0  0:03:45  0:03:06  0:00:39 10.7M 2208M    0     0  10.8M      0  0:03:45  0:03:23  0:00:22 10.6M     0  10.8M      0  0:03:45  0:0

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [ ]:
# Define image path here


In [ ]:
# Automated Mask (Instance Segmentation) Generation with SAM
import cv2
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
result = mask_generator.generate(image_rgb)


In [ ]:
#Annotate Masks with Supervision
import supervision as sv

mask_annotator = sv.MaskAnnotator(color_map = "index")
detections = sv.Detections.from_sam(result)
annotated_image = mask_annotator.annotate(image_bgr, detections)

In [ ]:
#Generate Segmentation Mask with Bounding Box

import cv2
from segment_anything import SamPredictor

mask_predictor = SamPredictor(sam)

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
mask_predictor.set_image(image_rgb)

box = np.array([70, 247, 626, 926])
masks, scores, logits = mask_predictor.predict(
    box=box,
    multimask_output=True
)